# parallel version of existed methods

In [1]:
import MDAnalysis as mda
from MDAnalysis import transformations as trans
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis

In [2]:
import dask
import dask.multiprocessing

dask.config.set(scheduler="processes")
from multiprocessing import cpu_count

n_jobs = cpu_count()

In [3]:
@dask.delayed
def run_block(blockslice, ana_base):
    """
    ana_base: object of AnalysisBase Class/Child Class
    """
    # universe.transfer_to_memory(start=blockslice.start, stop=blockslice.stop)
    ana_base.run(start=blockslice.start, stop=blockslice.stop, verbose=True)
    return ana_base

In [4]:
# load Universe
# dt in ps
u = mda.Universe("./input_data/interface.psf", "./input_data/trajectory.xyz", dt=0.025)
dim = [16.869, 16.869, 41.478, 90, 90, 120]
transform = trans.boxdimensions.set_dimensions(dim)
u.trajectory.add_transformations(transform)
u.transfer_to_memory(step=100)

In [5]:
# init AnalysisBase Class/Child Class

hbonds = HydrogenBondAnalysis(
    universe=u,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False,
)

In [6]:
# generate block
n_frames = u.trajectory.n_frames
n_blocks = n_jobs  #  it can be any realistic value (0 < n_blocks <= n_jobs)

n_frames_per_block = n_frames // n_blocks
blocks = [
    range(i * n_frames_per_block, (i + 1) * n_frames_per_block)
    for i in range(n_blocks - 1)
]
blocks.append(range((n_blocks - 1) * n_frames_per_block, n_frames))

In [7]:
blocks

[range(0, 3),
 range(3, 6),
 range(6, 9),
 range(9, 12),
 range(12, 15),
 range(15, 18),
 range(18, 21),
 range(21, 24),
 range(24, 27),
 range(27, 30),
 range(30, 33),
 range(33, 36),
 range(36, 39),
 range(39, 42),
 range(42, 45),
 range(45, 48),
 range(48, 51),
 range(51, 54),
 range(54, 57),
 range(57, 60),
 range(60, 63),
 range(63, 66),
 range(66, 69),
 range(69, 72),
 range(72, 75),
 range(75, 78),
 range(78, 81),
 range(81, 84),
 range(84, 87),
 range(87, 90),
 range(90, 93),
 range(93, 100)]

In [8]:
jobs = []
for bs in blocks:
    jobs.append(run_block(bs, hbonds))
jobs = dask.delayed(jobs)

In [9]:
results = jobs.compute()

In [10]:
print(results)

[<MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f2148ee0>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f26c82e0>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f2736ee0>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42cb778bb0>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f2287b80>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f23b97c0>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f2825400>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f28e8040>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b43131afc40>, <MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis object at 0x2b42f2364880>, <MDAnalysis.analysi

In [11]:
for tmp_ana_base in results:
    print(tmp_ana_base.results.hbonds.shape)
    print(tmp_ana_base.results.hbonds[0])

(983, 6)
[  0.           0.           1.         255.           2.70921881
 166.55259568]
(974, 6)
[  3.           0.           1.         198.           2.65063608
 155.77706696]
(969, 6)
[  6.           0.           1.         513.           2.92825451
 153.58767113]
(966, 6)
[  9.           0.           1.         513.           2.81667349
 171.6934334 ]
(955, 6)
[ 12.           0.           1.         513.           2.69279955
 169.08156004]
(942, 6)
[ 15.           0.           1.         513.           2.43899452
 160.60167188]
(929, 6)
[ 18.           0.           1.         513.           2.79956476
 170.16176062]
(945, 6)
[ 21.           0.           1.         513.           2.74496718
 156.04001763]
(905, 6)
[ 24.           3.           4.         525.           2.79053378
 165.0141109 ]
(904, 6)
[ 27.           0.           1.         375.           2.6898757
 160.80315588]
(915, 6)
[ 30.           0.           1.          84.           2.70264433
 157.42995399]
(918, 6)
[ 

# PMDA method

In [3]:
import MDAnalysis as mda
from MDAnalysis import transformations as trans
from pmda.hbond_analysis import HydrogenBondAnalysis as HBA

In [4]:
# load Universe
# dt in ps
u = mda.Universe("./input_data/interface.psf", "./input_data/trajectory.xyz", dt=0.025)
dim = [16.869, 16.869, 41.478, 90, 90, 120]
transform = trans.boxdimensions.set_dimensions(dim)
u.trajectory.add_transformations(transform)
u.transfer_to_memory(step=100)

In [5]:
hbonds = HBA(
    universe=u,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False,
)
hbonds.run()

/bigdata/jxzhu/205/software/pmda/pmda/parallel.py:342: UserWarning: Couldn't guess ideal number of blocks from scheduler. Setting n_blocks=1. Please provide `n_blocks` in call to method.
  warnings.warn(
/home/jxzhu/.conda/envs/band-alignment/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


In [8]:
hbonds.hbonds

array([[  0.        , 222.        , 224.        , 624.        ,
          2.88494294, 175.89718408],
       [  0.        , 267.        , 269.        , 372.        ,
          2.8500115 , 175.72556292],
       [  0.        , 273.        , 274.        , 288.        ,
          2.80944372, 168.00890198],
       ...,
       [ 99.        , 525.        , 526.        , 111.        ,
          2.68186619, 155.81009783],
       [ 99.        , 606.        , 607.        , 447.        ,
          2.97852013, 174.09155883],
       [ 99.        , 666.        , 667.        , 375.        ,
          2.5786764 , 154.1403947 ]])